# Doc2Vec으로 공시 사업보고서 유사도 계산하기

## 공시 사업 보고서 로드 및 전처리

In [ ]:
# dart.csv 파일 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT" -O dart.csv && rm -rf /tmp/cookies.txt

--2022-01-28 09:44:38--  https://docs.google.com/uc?export=download&confirm=EigQ&id=1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT
Resolving docs.google.com (docs.google.com)... 142.250.107.113, 142.250.107.100, 142.250.107.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.107.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-28-docs.googleusercontent.com/docs/securesc/r6hgqob8ju166rp3sqgln9ttl80ajns7/lmth1t9pqeg9em0rl1chhojt9b9leade/1643363025000/17609157229046208934/07371763786120897526Z/1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT?e=download [following]
--2022-01-28 09:44:39--  https://doc-0s-28-docs.googleusercontent.com/docs/securesc/r6hgqob8ju166rp3sqgln9ttl80ajns7/lmth1t9pqeg9em0rl1chhojt9b9leade/1643363025000/17609157229046208934/07371763786120897526Z/1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT?e=download
Resolving doc-0s-28-docs.googleusercontent.com (doc-0s-28-docs.googleusercontent.com)... 74.125.197.132, 2607:f8b0:400e:c03::84
Connec

In [ ]:
# 형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 22.81 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 70.6 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-01-28 09:44:49--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loc

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/dart.csv',  sep=',')
df = df.dropna()
df

,code,market,name,business
0,000020,KOSPI,동화약품,II. 사업의 내용\n1. 사업의 개요\n가. 일반적인 사항\n기업회계기준서 제11...
1,000040,KOSPI,KR모터스,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n수출주력시장인 유럽 불...
2,000050,KOSPI,경방,II. 사업의 내용\n1. 사업의 개요\n(1) 산업의 특성\n[섬유사업부문]\n면...
3,000060,KOSPI,메리츠화재,"II. 사업의 내용\n1. 사업의 개요\n가. 산업의 특성, 성장성, 경기변동의 특..."
4,000070,KOSPI,삼양홀딩스,"II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n지주회사(持株會社, H..."
...,...,...,...,...
2582,950130,KOSDAQ,엑세스바이오,II. 사업의 내용\n1. 사업의 개\n요\n당사는 체외진단 기술을 토대로 말라리아...
2583,950140,KOSDAQ,잉글우드랩,II. 사업의 내용\n1. 사업의 개요\n가. 화장품 ODM산업 개요\nODM은 O...
2584,950160,KOSDAQ,코오롱티슈진,II. 사업의 내용\n당사는 1999년 6월 9일 세포 유전자 기술 Platform...
2585,950170,KOSDAQ,JTC,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n(1) 산업의 특성\n...


In [ ]:
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row['business']
    tag = row['name']
    tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

    print('문서의 수 :', len(tagged_corpus_list))

  0%|          | 5/2295 [00:00<01:11, 31.85it/s]

문서의 수 : 1
문서의 수 : 2
문서의 수 : 3
문서의 수 : 4
문서의 수 : 5


  0%|          | 9/2295 [00:00<01:53, 20.10it/s]

문서의 수 : 6
문서의 수 : 7
문서의 수 : 8
문서의 수 : 9


  1%|          | 12/2295 [00:00<03:21, 11.34it/s]

...

문서의 수 : 2282
문서의 수 : 2283
문서의 수 : 2284
문서의 수 : 2285


100%|█████████▉| 2288/2295 [01:47<00:00, 20.94it/s]

문서의 수 : 2286
문서의 수 : 2287
문서의 수 : 2288
문서의 수 : 2289


100%|██████████| 2295/2295 [01:48<00:00, 21.19it/s]

문서의 수 : 2290
문서의 수 : 2291
문서의 수 : 2292
문서의 수 : 2293
문서의 수 : 2294
문서의 수 : 2295


In [ ]:
tagged_corpus_list[0]

TaggedDocument(words=['II', '.', '사업', '의', '내용', '1', '.', '사업', '의', '개요', '가', '.', '일반', '적', '인', '사항', '기업', '회계', '기준', '서', '제', '1110', '호', '"', '연결', '재무제표', '"', '의', '의하', '여', '2018', '년', '12', '월', '17', '일', '에', '설립', '한', '동화', '크립톤', '기업가', '정신', '제일', '호', '창업', '벤처', '전문', '사모', '투자', '합자회사', '를', '종속', '회사', '에', '편입', '하', '였', '습니다', '.', '나', '.', '지배', '기업', '의', '현황', '1', ')', '산업', '의', '특성', '제약', '산업', '은', '약학', ',', '화학', ',', '생물학', '등', '여러', '분야', '의', '지식', '과', '기술', '이', '복합', '적', '으로', '필요', '한', '기술', '·', '지식', '집약', '형', '산업', '으로', '인간', '의', '생명', '과', '보건', '에', '직접', '적', '으로', '관련', '된', '제품', '을', '생산', '하', '고', ',', '개발', '·', '제조', '·', '유통', '·', '가격', '등', '을', '정부', '에서', '엄격', '하', '게', '규제', '하', '고', '관리', '하', '는', '산업', '적', '특성', '을', '가지', '고', '있', '습니다', '.', '또한', '신약', '개발', '에', '는', '막대', '한', '시간', '과', '비용', '이', '소요', '되', '며', ',', '성공', '확률', '또한', '높', '지', '않', '으나', '신약', '개발', '에', '성공', '할', '경우', '엔', '엄청난

## Dec2Vec 학습 및 테스트



In [ ]:
from gensim.models import doc2vec

In [ ]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

Tag size: 2295 / 

In [ ]:
similar_doc = model.docvecs.most_similar('동화약품')
print(similar_doc)

[('종근당', 0.5611869096755981), ('일양약품', 0.5440576672554016), ('동아에스티', 0.5162414908409119), ('삼아제약', 0.5145184993743896), ('영진약품', 0.5068569779396057), ('삼일제약', 0.5034641027450562), ('제일약품', 0.4955061078071594), ('대웅제약', 0.48654472827911377), ('CMG제약', 0.48400360345840454), ('동국제약', 0.4834252595901489)]


In [ ]:
similar_doc = model.docvecs.most_similar('하이트진로')
print(similar_doc)

[('하이트진로홀딩스', 0.8578485250473022), ('보해양조', 0.5365418195724487), ('롯데칠성', 0.47363975644111633), ('무학', 0.45857664942741394), ('국순당', 0.4358399212360382), ('해마로푸드서비스', 0.41327327489852905), ('롯데제과', 0.3969845771789551), ('서부T&D', 0.37521135807037354), ('토니모리', 0.3741685748100281), ('삼양패키징', 0.3707728385925293)]


In [ ]:
similar_doc = model.docvecs.most_similar('LG이노텍')
print(similar_doc)

[('LG전자', 0.5360687971115112), ('LG', 0.5060262084007263), ('LG디스플레이', 0.5050520896911621), ('삼성전기', 0.49592745304107666), ('캠시스', 0.46597424149513245), ('서울반도체', 0.44433605670928955), ('서울바이오시스', 0.43775296211242676), ('이녹스첨단소재', 0.4066796898841858), ('매직마이크로', 0.4018380343914032), ('루멘스', 0.39737480878829956)]


In [ ]:
similar_doc = model.docvecs.most_similar('메리츠화재')
print(similar_doc)

[('메리츠금융지주', 0.7432562708854675), ('한화손해보험', 0.6948472857475281), ('DB손해보험', 0.6860089898109436), ('한화생명', 0.6792060136795044), ('흥국화재', 0.6647199392318726), ('롯데손해보험', 0.6520989537239075), ('현대해상', 0.6232735514640808), ('동양생명', 0.5883053541183472), ('코리안리', 0.5866270661354065), ('삼성화재', 0.5540577173233032)]


In [ ]:
similar_doc = model.docvecs.most_similar('카카오')
print(similar_doc)

[('네오위즈', 0.5006483793258667), ('네오위즈홀딩스', 0.49606674909591675), ('NAVER', 0.48792558908462524), ('카카오게임즈', 0.47018396854400635), ('퓨쳐스트림네트웍스', 0.45104604959487915), ('NEW', 0.4411781430244446), ('이스트소프트', 0.43794214725494385), ('컴투스', 0.43422752618789673), ('민앤지', 0.43036550283432007), ('다나와', 0.4298974275588989)]
